In [1]:
#imports
import lightgbm as lgb
import numpy as np
import pandas as pd
import sqlalchemy
from catboost import CatBoostRegressor
from hyperopt import hp
from hyperopt.pyll import scope
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from models.hyperopt_model import HyperoptModel
from models.utils import CustomDataFrameMapper
from models.utils import get_features

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# read the data
preprocessed_data = pd.read_pickle('./data/preprocesed_data.pd')

# Create a dataframe
df = pd.DataFrame(preprocessed_data)

In [3]:
# Create two new dataframes, one with the training rows, one with the test rows
train, test=train_test_split(df, test_size=0.2, random_state=0)

In [4]:
# Show the number of observations for the test and training dataframes
print('Number of observations in the training data:', len(train))
print('Number of observations in the test data:',len(test))

Number of observations in the training data: 9303
Number of observations in the test data: 2326


In [5]:
    #some parameters
    DO_LOWESS = False
    hyperopt = True
    # Run LGBMRegressor with hyperopt optimization
    lgbm = HyperoptModel(train.copy(), test.copy(), 'lgbm', cv=5)
    comment_features, mention_features, post_features, user_features = get_features()
    lgbm.raw_features = []
    lgbm.pipeline = Pipeline([
        ('prepare_features', FeatureUnion([
            ('user_stats', user_features),
            ('mention_stats', mention_features),
            ('posts_stats', post_features),
            ('comments_stats', comment_features)
        ])),
        ('estimate', lgb.LGBMRegressor())
    ])
    for transformer in lgbm.pipeline.named_steps['prepare_features'].transformer_list:
        lgbm.raw_features += [t[0] if isinstance(t[0], str) else t[0][0] for t in transformer[1].features]
   
    """ find number of trees """
    # num_trees_train, num_trees_eval = train_test_split(train, test_size=0.2, random_state=0)
    # X = num_trees_train[list(filter(lambda column: column in lgbm.raw_features, lgbm.train.columns))]
    # y = num_trees_train['score']
    #
    # eval_X = num_trees_eval[list(filter(lambda column: column in lgbm.raw_features, lgbm.train.columns))]
    # eval_X = lgbm.pipeline.named_steps['prepare_features'].fit_transform(eval_X)
    # eval_set = (eval_X, num_trees_eval['score'])
    #
    # best = lgbm.pipeline.fit(X=X, y=y, estimate__eval_set=eval_set, estimate__early_stopping_rounds=10)
    # print(best.named_steps['estimate'].best_iteration)

    lgbm.space = {
        'estimate__objective': hp.choice('estimate__objective', ['regression']),
        'estimate__n_estimators': hp.choice('estimate__n_estimators', [400]),
        'estimate__seed': hp.choice('estimate__seed', [0]),

        'estimate__learning_rate': hp.loguniform('estimate__learning_rate', -7, 0),
        'estimate__num_leaves': scope.int(hp.qloguniform('estimate__num_leaves', 1, 7, 1)),
        'estimate__feature_fraction': hp.uniform('estimate__feature_fraction', 0.5, 1),
        'estimate__bagging_fraction': hp.uniform('estimate__bagging_fraction', 0.5, 1),
        'estimate__min_data_in_leaf': scope.int(hp.qloguniform('estimate__min_data_in_leaf', 0, 6, 1)),
        'estimate__min_sum_hessian_in_leaf': hp.loguniform('estimate__min_sum_hessian_in_leaf', -16, 5),
        'estimate__lambda_l1': hp.choice('lambda_l1', [0, hp.loguniform('estimate__lambda_l1_positive', -16, 2)]),
        'estimate__lambda_l2': hp.choice('lambda_l2', [0, hp.loguniform('estimate__lambda_l2_positive', -16, 2)]),
    }

    if hyperopt:
        lgbm.run(do_lowess=DO_LOWESS)
    else:
        # train with default params
        lgbm.pipeline.fit(X=lgbm.X_train, y=lgbm.y_train)
        lgbm.model = lgbm.pipeline
        lgbm.stats()
        lgbm.plot_predicted_vs_actual()
        lgbm.plot_feature_importance()

Performing parameters optimization...
[1/50]	cv_eval_time=12.62 sec	RMSE=0.087764	R^2=0.625503
[2/50]	cv_eval_time=44.39 sec	RMSE=0.068023	R^2=0.774703
[3/50]	cv_eval_time=11.86 sec	RMSE=0.089884	R^2=0.606522
[4/50]	cv_eval_time=18.93 sec	RMSE=0.073356	R^2=0.740015
[5/50]	cv_eval_time=34.09 sec	RMSE=0.104866	R^2=0.465882
[6/50]	cv_eval_time=50.76 sec	RMSE=0.097119	R^2=0.540366
[7/50]	cv_eval_time=11.42 sec	RMSE=0.083055	R^2=0.661602
[8/50]	cv_eval_time=47.26 sec	RMSE=0.074657	R^2=0.728273
[9/50]	cv_eval_time=15.90 sec	RMSE=0.118731	R^2=0.313019
[10/50]	cv_eval_time=52.75 sec	RMSE=0.105831	R^2=0.443711
[11/50]	cv_eval_time=20.31 sec	RMSE=0.082299	R^2=0.670907
[12/50]	cv_eval_time=34.27 sec	RMSE=0.073296	R^2=0.737763
[13/50]	cv_eval_time=47.50 sec	RMSE=0.069833	R^2=0.764215
[14/50]	cv_eval_time=11.76 sec	RMSE=0.120417	R^2=0.292921
[15/50]	cv_eval_time=12.07 sec	RMSE=0.071680	R^2=0.750777
[16/50]	cv_eval_time=15.64 sec	RMSE=0.090164	R^2=0.604867
[17/50]	cv_eval_time=13.00 sec	RMSE=0.07008